In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install llama-index==0.10.30 sentence-transformers faiss-cpu


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever


In [ ]:
documents = SimpleDirectoryReader(
    input_dir="data/rti_docs",
    recursive=True
).load_data()

print(f"Loaded {len(documents)} documents")


In [ ]:
splitter = SentenceSplitter(chunk_size=400, chunk_overlap=50)
nodes = splitter.get_nodes_from_documents(documents)

print(f"Created {len(nodes)} chunks")


In [ ]:
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex(
    nodes,
    embed_model=embed_model
)

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3
)

print("Index built successfully")


In [ ]:
RTI_TEMPLATE = """
To,
The Public Information Officer,
{authority}

Subject: Request for information under Section 6(1) of RTI Act, 2005

Sir/Madam,

I seek the following information:

{questions}

Yours faithfully,
{applicant_name}
"""


In [ ]:
def generate_rti(user_query, authority, applicant_name):
    
    retrieved = retriever.retrieve(user_query)
    
    if not retrieved:
        return "❌ No legal basis found. RTI draft cannot be generated."
    
    questions = []
    
    for i, node in enumerate(retrieved, start=1):
        line = node.text.strip().split(".")[0]
        questions.append(f"{i}. {line}")
    
    question_text = "\n".join(questions)
    
    return RTI_TEMPLATE.format(
        authority=authority,
        questions=question_text,
        applicant_name=applicant_name
    )


In [ ]:
draft = generate_rti(
    user_query="details of pothole repairs in my ward",
    authority="Municipal Corporation of Thane",
    applicant_name="Prachiti Parab"
)

print(draft)


In [ ]:
draft = generate_rti(
    user_query="quantum teleportation policy of municipality",
    authority="Municipal Corporation of Thane",
    applicant_name="Prachiti Parab"
)

print(draft)
